# Imports

In [20]:
import time
time_start_notebook = time.time()

In [21]:
import os
import sys
import numpy as np
import pandas as pd
from pathlib import Path

from langchain.text_splitter import CharacterTextSplitter
import pypdf
from PyPDF2 import PdfReader
from pypdf import PdfWriter
from pdf2image import convert_from_path, convert_from_bytes
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter


from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.embeddings import OpenAIEmbeddings,AzureOpenAIEmbeddings
from langchain.vectorstores import FAISS

from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

import streamlit
import streamlit as st
import base64
import os
import glob

import streamlit as st
from dotenv import load_dotenv
from werkzeug.utils import secure_filename
import os
import sys
import glob

import openai
import langchain
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import FAISS

from langchain.llms import AzureOpenAI
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

#--- ocr
import pdf2image
from pdf2image import convert_from_path, convert_from_bytes
import pytesseract

In [22]:
%load_ext watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [23]:
!pip --version

pip 24.0 from C:\Users\a126291\venv\py311_camelot\Lib\site-packages\pip (python 3.11)



In [24]:
%watermark -iv

langchain  : 0.1.16
sys        : 3.11.7 (tags/v3.11.7:fa7a6f2, Dec  4 2023, 19:24:49) [MSC v.1937 64 bit (AMD64)]
streamlit  : 1.33.0
openai     : 1.22.0
pdf2image  : 1.17.0
pypdf      : 4.2.0
numpy      : 1.26.4
pandas     : 1.5.3
pytesseract: 0.3.10



# Config

In [25]:
from pathlib import Path
path_pdf_ocr = r"C:\Users\a126291\OneDrive - AmerisourceBergen(ABC)\data\pdf_files\ocr_pdf\screenshot_pages_eylea.pdf"
path_pdf_non_ocr = r"C:\Users\a126291\OneDrive - AmerisourceBergen(ABC)\data\pdf_files\fda_drugs\fda_eylea.pdf"

# source directory

In [26]:
import shutil
if os.path.isdir('src'):
    shutil.rmtree('src')

In [27]:
import os
if not os.path.isdir('src'):
    os.makedirs('src')

In [28]:
with open('src/__init__.py','w') as fh:
    fh.write('# Author: Bhishan Poudel\n')

In [29]:
!ls src

__init__.py


In [30]:
!cat src/__init__.py

# Author: Bhishan Poudel


# util: config

In [31]:
%%writefile src/config.py

def get_config():
    # config
    openai_api_key = "f4001c4b49a845309d386e9014b7ae6b"
    openai_api_base = "https://qa.gai.cencora.com/aoai"
    azure_endpoint = openai_api_base
    openai_api_version = "2023-03-15-preview"
    openai_api_type = "azure"

    openai_deployment_name = "gpt-35-turbo-16k"
    openai_model_name      = "gpt-35-turbo-16k"

    # os environments
    import os
    os.environ['OPENAI_API_TYPE'] = openai_api_type
    os.environ['OPENAI_API_VERSION'] = openai_api_version
    os.environ['OPENAI_API_BASE'] = openai_api_base
    os.environ['OPENAI_API_KEY'] = openai_api_key

    # openai environments
    import openai
    openai.api_key = openai_api_key
    openai.api_type = openai_api_type
    openai.api_base = openai_api_base
    openai.api_version = openai_api_version
    openai.api_key = openai_api_key

    config = {
    'openai_api_base': openai_api_base,
    'openai_api_version': openai_api_version,
    'deployment_name': openai_deployment_name,
    'model_name': openai_model_name,
    'openai_api_key': openai_api_key,
    'openai_api_type': openai_api_type
    }

    return config

config = get_config()

Writing src/config.py


In [32]:
from src.config import config

config['model_name']

'gpt-35-turbo-16k'

# Util: ocr

In [33]:
%%writefile src/util_ocr.py
import streamlit as st
import os

#--- ocr
import pdf2image
import pytesseract
pytesseract.tesseract_cmd = r"C:\Users\a126291\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"

#@st.cache_data
def get_ocr_lst_text_from_uploaded_file(uploaded_file, language='eng',pytesseract_path=None):
    import pdf2image
    import pytesseract
    from pathlib import Path

    if pytesseract_path == None:
        pytesseract_path = r"C:\Users\a126291\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"
    pytesseract.tesseract_cmd = pytesseract_path

    try: # streamlit uploded file
        data_bytes = uploaded_file.getvalue()
    except:
        try: # regular file path
            path_pdf = Path(uploaded_file)
            if path_pdf.exists():
                with open(path_pdf, 'rb') as file:
                    data_bytes = file.read()
        except:
            pass

    pil_images = pdf2image.convert_from_bytes(data_bytes)
    lst_txt = []
    for pil_image in pil_images:
        text_one_page = pytesseract.image_to_string(pil_image, lang=language)
        lst_txt.append(text_one_page)
    return lst_txt

Writing src/util_ocr.py


In [34]:
from src.util_ocr import get_ocr_lst_text_from_uploaded_file

path_pdf = path_pdf_ocr
lst_text = get_ocr_lst_text_from_uploaded_file(uploaded_file=path_pdf, language='eng',pytesseract_path=None)

print(lst_text[0][-500:])

croliters) admuistered by
intravitreal injection every 4 weeks (monthly) for the frst 5 imjections, followed by 2 mg

(0.05 mL) via intravitreal injection once every 8 weeks (2 months). Although EYLEA may be
dosed as frequently as 2 mg every 4 weeks (monthly), additional efficacy was not demonstrated
10 most patients when EYLEA was dosed every 4 weeks compared to every 8 weeks [see
Clinical Studies (14.4)]. Some patients may need every 4 week (monthly) dosing after the firs
20 weeks (5 months).



# Util: Text

In [35]:
%%writefile src/util_text.py

#========================== util: text
from langchain.text_splitter import CharacterTextSplitter
from PyPDF2 import PdfReader
from pypdf import PdfWriter
from pdf2image import convert_from_path, convert_from_bytes
import os
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# local imports
from src.util_ocr import get_ocr_lst_text_from_uploaded_file

# import pytesseract
import pytesseract
pytesseract.tesseract_cmd = r"C:\Users\a126291\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"

# Single PDF No OCR
def get_lst_text_from_single_pdf_no_ocr(path_pdf):
    r'Get list of text for each pages of pdf document'
    from PyPDF2 import PdfReader

    pdf_reader = PdfReader(path_pdf)
    lst_text = []
    for page in pdf_reader.pages:
        text = page.extract_text()
        lst_text.append(text)
    return lst_text


def get_lst_text_from_uploaded_file(uploaded_file,ocr=False,pytesseract_path=None):
    r'''Get list of text from a single pdf (works for both ocr and non-ocr pdf).

    '''
    lst_text = []
    if ocr:
        lst_text = get_ocr_lst_text_from_uploaded_file(uploaded_file, language='eng',pytesseract_path=None)
    else:
        lst_text = get_lst_text_from_single_pdf_no_ocr(uploaded_file)
    return lst_text


Writing src/util_text.py


In [36]:
# !explorer.exe .

In [37]:
from src.util_text import get_lst_text_from_uploaded_file
uploaded_file = path_pdf_ocr
lst_text = get_lst_text_from_uploaded_file(uploaded_file,ocr=True,pytesseract_path=None)

In [38]:
len(lst_text)

6

In [39]:
if lst_text:
    print(lst_text[0])

y DOSAGE AND ADMINISTRATION

21 Important Injection Instructions

For ophthalmic intravitreal injection. EYLEA must only be admumstered by a qualified
physician.

2.2 — Neovascular (Wet) Age-Related Macular Degeneration (AMD)

The recommended dose for EYLEA 1s 2 me (0.05 mL or 50 microliters) admuistered by
intravitreal injection every 4 weeks (monthly) for the first 12 weeks (3 months), followed by

2 me (0.05 mL) via intravitreal injection once every 8 weeks (2 months). Although EYLEA may
be dosed as frequently as 2 me every 4 weeks (monthly), additional efficacy was not
demonstrated in most patients when EYLEA was dosed every 4 weeks compared to every

8 weeks [see Clinical Studies (14.1)]. Some patients may need every 4 week (monthly) dosing
after the first 12 weeks (3 months).

13 Macular Edema Following Retinal Vein Occlusion (RVO)

The recommended dose for EYLEA 1s 2 mg (0.05 ml or 50 microliters) admumstered by
intravitreal injection once every 4 weeks (monthly) [see Clinical S

# Util: doc

In [40]:
%%writefile src/util_doc.py

import os
import sys
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

# local imports
from src.util_text import get_lst_text_from_uploaded_file

def get_doc_chunks_from_lst_text(lst_text,path_pdf):
    """Convert list of text to list of langchain Documents.

    Note: we have chunks, not the page documents

    from langchain.docstore.document import Document
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    """
    from langchain.docstore.document import Document
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from pathlib import Path

    if isinstance(lst_text, str):
        # Take a single string as one page
        lst_text = [lst_text]

    # get docs
    page_docs = [Document(page_content=page) for page in lst_text]

    # Add page numbers as metadata
    for i, doc in enumerate(page_docs):
        doc.metadata["page"] = i + 1

    # Split pages into chunks
    doc_chunks = []

    for doc in page_docs:
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=2000,
            separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
            chunk_overlap=0,
        )
        chunks = text_splitter.split_text(doc.page_content)
        for i, chunk in enumerate(chunks):
            doc = Document(
                page_content=chunk, metadata={"page": doc.metadata["page"], "chunk": i}
            )
            # Add sources a metadata
            doc.metadata["source"] = f"{doc.metadata['page']}-{doc.metadata['chunk']}"
            doc.metadata["path_name"] = Path(path_pdf).name
            doc.metadata["path_name_full"] = str(path_pdf)
            doc_chunks.append(doc)
    return doc_chunks

def get_all_doc_chunks_from_path_pdfs(path_pdfs,ocr=False):
    r'''Get document chunks for all the pdfs.

    Depends:
    ========
    - get_lst_text_from_uploaded_file
    - get_doc_chunks_from_lst_text

    '''
    all_doc_chunks = []
    for i, path_pdf in enumerate(path_pdfs):
        lst_text =  get_lst_text_from_uploaded_file(path_pdf,ocr=ocr)
        doc_chunks = get_doc_chunks_from_lst_text(lst_text,path_pdf)
        for doc in doc_chunks:
            doc.metadata["pdf_number"] = str(i+1)
            all_doc_chunks.append(doc)
    return all_doc_chunks

Writing src/util_doc.py


In [41]:
from src.util_doc import get_all_doc_chunks_from_path_pdfs

path_pdfs = [path_pdf_ocr]

all_doc_chunks = get_all_doc_chunks_from_path_pdfs(path_pdfs,ocr=True)
doc_chunk = all_doc_chunks[0]

type(doc_chunk)

langchain_core.documents.base.Document

In [42]:
doc_chunk

Document(page_content='y DOSAGE AND ADMINISTRATION\n\n21 Important Injection Instructions\n\nFor ophthalmic intravitreal injection. EYLEA must only be admumstered by a qualified\nphysician.\n\n2.2 — Neovascular (Wet) Age-Related Macular Degeneration (AMD)\n\nThe recommended dose for EYLEA 1s 2 me (0.05 mL or 50 microliters) admuistered by\nintravitreal injection every 4 weeks (monthly) for the first 12 weeks (3 months), followed by\n\n2 me (0.05 mL) via intravitreal injection once every 8 weeks (2 months). Although EYLEA may\nbe dosed as frequently as 2 me every 4 weeks (monthly), additional efficacy was not\ndemonstrated in most patients when EYLEA was dosed every 4 weeks compared to every\n\n8 weeks [see Clinical Studies (14.1)]. Some patients may need every 4 week (monthly) dosing\nafter the first 12 weeks (3 months).\n\n13 Macular Edema Following Retinal Vein Occlusion (RVO)\n\nThe recommended dose for EYLEA 1s 2 mg (0.05 ml or 50 microliters) admumstered by\nintravitreal injection

# Util: LLM

In [57]:
import os
import json
import yaml
import numpy as np
import pandas as pd

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import AzureChatOpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
import pandas as pd


# Variables
path_pdf = r"C:\Users\a126291\OneDrive - AmerisourceBergen(ABC)\data\pdf_files\cencora_earning_reports\AmerisourceBergen-Reports-Fiscal-2023-Second-Quarter-Results-2023.pdf"

embed_model= "text-embedding-ada-002"
vectore_store_persist_directory = 'vector_store'

# Config
import sys
import os
sys.path.append(os.path.expanduser("~/.utils"))
from util_openai import get_config_dict
config = get_config_dict(env=None)

# openai_api_key="242ad9b0c6d84b5fb4307061d5f05a86"
# openai_api_version="2023-03-15-preview"

# openai_api_base="https://cencoragaidev-bhakadc4hreqbthb.z01.azurefd.net/gpsdatascience_dev"
# azure_endpoint="https://cencoragaidev-bhakadc4hreqbthb.z01.azurefd.net/gpsdatascience_dev"
deployment_name='gpt-35-turbo-16k'

openai_api_key = config["openai_api_key"]
openai_api_version = config["openai_api_version"]

openai_api_base = config["openai_api_base"]
azure_endpoint = config["azure_endpoint"]
# deployment_name = config["deployment_name"]

# Load pdf
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(path_pdf)
documents = loader.load()

# Split Documents Into Chunks
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

# embedding
from langchain.embeddings import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    chunk_size=1,
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    )

# vector store
from langchain.vectorstores import FAISS

pages = loader.load_and_split()
vector_store = FAISS.from_documents(documents=pages, embedding=embeddings)
vector_store.save_local(vectore_store_persist_directory)

# llm
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    openai_api_type="azure",
    openai_api_key=openai_api_key,
    deployment_name=deployment_name,
    model_name="text-davinci-002",
    azure_endpoint=azure_endpoint,
    openai_api_version=openai_api_version,
    temperature=0
    )

# Get vector store from local file
from langchain.vectorstores import FAISS

vector_store = FAISS.load_local(vectore_store_persist_directory, embeddings, allow_dangerous_deserialization=True,)
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":2},
    return_source_documents=True,
)

# Create RetrivalQA Chain
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever = retriever
)

# Create Query And Get Response
query = 'What are the year to year gain in revenue?'
result = qa.invoke({'query': query})
print(result['result'])
print('='*100)

## Using: ConversationalRetrievalChain
from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(llm,retriever)
chat_history = []
query = "Is there gain or loss of revenue in this quarter?"

result = qa.invoke({'question': query, 'chat_history': chat_history})
print(result['answer'])
print('='*100)
chat_history = [(query,result['answer'])]
query = "Is there gain in revenue for U.S. Healthcare Solutions this quarter?"

result = qa.invoke({'question': query, 'chat_history': chat_history})
print(result['answer'])

C:\Users\a126291\venv\py311_camelot\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.azure_openai.AzureOpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(
C:\Users\a126291\venv\py311_camelot\Lib\site-packages\langchain_community\embeddings\azure_openai.py:113: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://qa.gai.cencora.com/aoai to https://qa.gai.cencora.com/aoai/openai.
  warnings.warn(
C:\Users\a126291\venv\py311_camelot\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationW

ValidationError: 1 validation error for AzureChatOpenAI
__root__
  base_url and azure_endpoint are mutually exclusive (type=value_error)